Import Dependencies 

In [1]:
# !git clone https://github.com/tmengel/Deep-Learning.git
# %cd Deep-Learning/finalProject
# %pip install uproot
# %pip install pandas

# from google.colab import drive
# drive.mount('/content/drive')

import pulsenet as pn
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import pandas as pd

2023-05-11 00:01:17.464122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 00:01:18.851073: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tmengel/root/lib
2023-05-11 00:01:18.851334: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-11 00:01:20.646150: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared obj

File name

In [2]:
filename = "ysoTracesWithPileup.root"
# filename = "/content/drive/MyDrive/DeepLearningFinalProject/ysoTracesWithPileup.root"

discriminator_head_weights = "weights/discriminator_head_initial.h5"
discriminator_base_weights = "weights/discriminator_base_initial.h5"
classifier_head_weights = "weights/classifier_head_initial.h5"
classifier_base_weights = "weights/classifier_base_initial.h5"
phase_weights = "weights/phase_initial.h5"
amplitude_weights = "weights/amplitude_initial.h5"
transfer_head_weights = "weights/transfer_head_initial.h5"

discriminator_base_fine_tuned_weights = "weights/discriminator_base_fine_tuned.h5"
classifier_base_fine_tuned_weights = "weights/classifier_base_fine_tuned.h5"
phase_fine_tuned_weights = "weights/phase_fine_tuned.h5"
amplitude_fine_tuned_weights = "weights/amplitude_fine_tuned.h5"
transfer_head_fine_tuned_weights = "weights/transfer_head_fine_tuned.h5"

discriminator_initial_history ="history/discriminator_initial_history.h5"
classifier_initial_history = "history/classifier_initial_history.h5"
phase_initial_history = "history/phase_initial_history.h5"
amplitude_initial_history = "history/amplitude_initial_history.h5"

discriminator_base_control_weights = "weights/discriminator_base_control.h5"
classifier_base_control_weights = "weights/classifier_base_control.h5"
phase_control_weights = "weights/phase_control.h5"
amplitude_control_weights = "weights/amplitude_control.h5"
transfer_head_control_weights = "weights/transfer_head_control.h5"

transfer_history = "history/transfer_history.h5"
fine_tune_history = "history/fine_tune_history.h5"
control_history = "history/control_history.h5"

transfer_model = "weights/transfer_model.h5"
fine_tuned_model = "weights/fine_tuned_model.h5"
control_model = "weights/control_model.h5"

In [3]:
# Train Discriminator
# Get Data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# output files
historyfile = "history/discriminator_initial_history.h5"

# Create model
model = keras.Sequential([
    pn.TraceDiscriminatorBase(name="discriminator_base"),
    pn.TraceDiscriminatorHead(name="discriminator_head")
])
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_trace, epochs=10, batch_size=256, validation_split=0.2, verbose=1)
model.layers[0].save_weights(discriminator_base_weights)
model.layers[1].save_weights(discriminator_head_weights)
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(discriminator_initial_history, key="hist")

!cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
!cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

Created 261852 samples: 50.17108901211371 % pileup, 49.82891098788629 % no pileup
Epoch 1/10
450/819 [===============>..............] - ETA: 39s - loss: 0.0041 - accuracy: 0.1582

KeyboardInterrupt: 

In [ ]:
# Train Classifier
# Get Data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
y_pileup = pn.EncodePileup(y_phase)
# Create model
model = keras.Sequential([
    pn.TraceClassifierBase(name="classifier_base"),
    pn.TraceClassifierHead(name="classifier_head")
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_pileup, epochs=200, batch_size=128, validation_split=0.2, verbose=1)
model.layers[0].save_weights(classifier_base_weights)
model.layers[1].save_weights(classifier_head_weights)
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(classifier_initial_history, key="hist")

!cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
!cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

In [ ]:
# Train Amplitude Regressor
# Get Data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# Create model
model = keras.Sequential([
    pn.TraceAmplitudeRegressor(name = "amplitude_regressor")
])
model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_amp, epochs=200, batch_size=128, validation_split=0.2, verbose=1)
model.layers[0].save_weights(amplitude_weights)
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(amplitude_initial_history, key="hist")

!cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
!cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

In [ ]:
# Train Phase Regressor
# Get Data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# Create model
model = keras.Sequential([
    pn.TracePhaseRegressor(name = "phase_regressor")
])
model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_phase, epochs=200, batch_size=128, validation_split=0.2, verbose=1)
model.layers[0].save_weights(phase_weights)
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(phase_initial_history, key="hist")

!cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
!cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

In [3]:
input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
classifer_feature_vec = pn.TraceClassifierBase(name = "classifier_base")(input)
discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
trace_output = pn.TraceClassifierDiscriminatorHead(name = "transfer_head")([discriminator_feature_vec, classifer_feature_vec])
phase_output = pn.TracePhaseRegressor(name="phase_regressor")(trace_output)
amplitude_output = pn.TraceAmplitudeRegressor(name ="amplitude_regressor")(trace_output)
classifier_output = pn.TraceClassifierHead(name="classifier_head")(classifer_feature_vec)

In [4]:
# load new data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# create model
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer='adam', loss=['mse', 'mse', 'mse'], metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights(discriminator_base_weights)
model.layers[model_names.index("classifier_base")].load_weights(classifier_base_weights)
model.layers[model_names.index("phase_regressor")].load_weights(phase_weights)
model.layers[model_names.index("amplitude_regressor")].load_weights(amplitude_weights)
# freeze layers
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("classifier_base")].trainable = False
model.layers[model_names.index("phase_regressor")].trainable = False
model.layers[model_names.index("amplitude_regressor")].trainable = False                           
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, [y_trace, y_phase, y_amp], epochs=200, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("transfer_head")].save_weights(transfer_head_weights)
# save model
model.save(transfer_model)
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(transfer_history, key="hist")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 300)]     0           []                               
                                                                                                  
 discriminator_base (TraceDiscr  (None, 2, 300)      992358      ['input_1[0][0]']                
 iminatorBase)                                                                                    
                                                                                                  
 classifier_base (TraceClassifi  (None, 1, 300)      244456      ['input_1[0][0]']                
 erBase)                                                                                          
                                                                                              

In [ ]:
# load new data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# create model
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=['mse', 'mse', 'mse'], metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights(discriminator_base_weights)
model.layers[model_names.index("classifier_base")].load_weights(classifier_base_weights)
model.layers[model_names.index("phase_regressor")].load_weights(phase_weights)
model.layers[model_names.index("amplitude_regressor")].load_weights(amplitude_weights)
model.layers[model_names.index("transfer_head")].load_weights(transfer_head_weights)                     
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, [y_trace, y_phase, y_amp], epochs=200, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("discriminator_base")].save_weights(discriminator_base_fine_tuned_weights)
model.layers[model_names.index("classifier_base")].save_weights(classifier_base_fine_tuned_weights)
model.layers[model_names.index("phase_regressor")].save_weights(phase_fine_tuned_weights)
model.layers[model_names.index("amplitude_regressor")].save_weights(amplitude_fine_tuned_weights)
model.layers[model_names.index("transfer_head")].save_weights(transfer_head_fine_tuned_weights)
# save model
model.save(fine_tuned_model)
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(fine_tune_history, key="hist")

In [ ]:
# load new data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# create model
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=['mse', 'mse', 'mse'], metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# print summary
model.summary()
# train control model
history = model.fit(x_trace, [y_trace, y_phase, y_amp], epochs=400, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("discriminator_base")].save_weights(discriminator_base_control_weights)
model.layers[model_names.index("classifier_base")].save_weights(classifier_base_control_weights)
model.layers[model_names.index("phase_regressor")].save_weights(phase_control_weights)
model.layers[model_names.index("amplitude_regressor")].save_weights(amplitude_control_weights)
model.layers[model_names.index("transfer_head")].save_weights(transfer_head_control_weights)
# save model
model.save(control_model)
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(control_history, key="hist")

In [ ]:
transfer_head_weights_trace ="weights/transfer_head_trace.h5"
transfer_history_trace = "history/transfer_history_trace.h5"

transfer_head_long_transfer_weights = "weights/transfer_head_long_tranfer.h5"
transfer_history_long_transfer = "history/transfer_history_long_transfer.h5"
transfer_model_long_transfer = "weights/transfer_model_long_transfer.h5"

discriminator_base_fine_tuned_weights_long_transfer = "weights/discriminator_base_fine_tuned_weights_long_transfer.h5"
classifier_base_fine_tuned_weights_long_transfer = "weights/classifier_base_fine_tuned_weights_long_transfer.h5"
phase_regressor_fine_tuned_weights_long_transfer = "weights/phase_regressor_fine_tuned_weights_long_transfer.h5"
amplitude_regressor_fine_tuned_weights_long_transfer = "weights/amplitude_regressor_fine_tuned_weights_long_transfer.h5"
transfer_head_fine_tuned_weights_long_transfer = "weights/transfer_head_fine_tuned_weights_long_transfer.h5"

fine_tuned_long_transfer_model = "weights/fine_tuned_long_transfer_model.h5"
fine_tuned_long_transfer_history = "history/fine_tuned_long_transfer_history.h5"

In [ ]:
# load new data
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# create model
model = models.Model(inputs=input, outputs=trace_output)
model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights(discriminator_base_weights)
model.layers[model_names.index("classifier_base")].load_weights(classifier_base_weights)
# freeze layers
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("classifier_base")].trainable = False                         
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace,y_trace, epochs=200, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("transfer_head")].save_weights(transfer_head_weights_trace)
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(transfer_history_trace, key="hist")

In [ ]:
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# create model
model = models.Model(inputs=input, outputs=[phase_output, amplitude_output])
model.compile(optimizer='adam', loss=['mse', 'mse'], metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights(discriminator_base_weights)
model.layers[model_names.index("classifier_base")].load_weights(classifier_base_weights)
model.layers[model_names.index("phase_regressor")].load_weights(phase_weights)
model.layers[model_names.index("amplitude_regressor")].load_weights(amplitude_weights)
model.layers[model_names.index("transfer_head")].load_weights(transfer_head_weights_trace)
# freeze layers
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("classifier_base")].trainable = False
model.layers[model_names.index("phase_regressor")].trainable = False
model.layers[model_names.index("amplitude_regressor")].trainable = False                           
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, [y_phase, y_amp], epochs=200, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("transfer_head")].save_weights(transfer_head_long_transfer_weights)
# save model
model.save(transfer_model_long_transfer)
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(transfer_history_long_transfer, key="hist")

In [ ]:
x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
# create model
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=['mse', 'mse', 'mse'], metrics=['accuracy'])
# load weights
model.layers[model_names.index("discriminator_base")].load_weights(discriminator_base_weights)
model.layers[model_names.index("classifier_base")].load_weights(classifier_base_weights)
model.layers[model_names.index("phase_regressor")].load_weights(phase_weights)
model.layers[model_names.index("amplitude_regressor")].load_weights(amplitude_weights)
model.layers[model_names.index("transfer_head")].load_weights(transfer_head_long_transfer_weights)                      
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, [y_trace, y_phase, y_amp], epochs=200, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("transfer_head")].save_weights(transfer_head_fine_tuned_weights_long_transfer)
model.layers[model_names.index("discriminator_base")].save_weights(discriminator_base_fine_tuned_weights_long_transfer)
model.layers[model_names.index("classifier_base")].save_weights(classifier_base_fine_tuned_weights_long_transfer)
model.layers[model_names.index("phase_regressor")].save_weights(phase_regressor_fine_tuned_weights_long_transfer)
model.layers[model_names.index("amplitude_regressor")].save_weights(amplitude_regressor_fine_tuned_weights_long_transfer)

# save model
model.save(fine_tuned_long_transfer_model)
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf(fine_tuned_long_transfer_history, key="hist")

In [3]:
# load new data
input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
trace_output = pn.TraceDiscriminatorHead(name = "discriminator_head")(discriminator_feature_vec)
phase_output = pn.TracePhaseRegressor(name="phase_regressor")(trace_output)
amplitude_output = pn.TraceAmplitudeRegressor(name ="amplitude_regressor")(trace_output)

x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)

# Train Discriminator
# create model
model = models.Model(inputs=input, outputs=trace_output)
model.compile(optimizer="adam", loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# print summary
model.summary()
# train model
history = model.fit(x_trace, y_trace, epochs=500, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("discriminator_base")].save_weights("weights/sequential_training_discriminator_base_weights.h5")
model.layers[model_names.index("discriminator_head")].save_weights("weights/sequential_training_discriminator_head_weights.h5")
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf("history/sequential_training_discriminator_history.h5", key="hist")

# Train Phase Regressor
model = models.Model(inputs=input, outputs=phase_output)
model.compile(optimizer="adam", loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
model.layers[model_names.index("discriminator_base")].load_weights("weights/sequential_training_discriminator_base_weights.h5")
model.layers[model_names.index("discriminator_head")].load_weights("weights/sequential_training_discriminator_head_weights.h5")
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("discriminator_head")].trainable = False
# print summary
model.summary()
history = model.fit(x_trace, y_phase, epochs=500, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("phase_regressor")].save_weights("weights/sequential_training_phase_regressor_weights.h5")
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf("history/sequential_training_phase_history.h5", key="hist")

# Train Amplitude Regressor
model = models.Model(inputs=input, outputs=amplitude_output)
model.compile(optimizer="adam", loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
model.layers[model_names.index("discriminator_base")].load_weights("weights/sequential_training_discriminator_base_weights.h5")
model.layers[model_names.index("discriminator_head")].load_weights("weights/sequential_training_discriminator_head_weights.h5")
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("discriminator_head")].trainable = False
# print summary
model.summary()
history = model.fit(x_trace, y_amp, epochs=500, batch_size=128, validation_split=0.2, verbose=1)
# save weights
model.layers[model_names.index("amplitude_regressor")].save_weights("weights/sequential_training_amplitude_regressor_weights.h5")
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf("history/sequential_training_amplitude_history.h5", key="hist")

#fine tune model
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
model.layers[model_names.index("discriminator_base")].load_weights("weights/sequential_training_discriminator_base_weights.h5")
model.layers[model_names.index("discriminator_head")].load_weights("weights/sequential_training_discriminator_head_weights.h5")
model.layers[model_names.index("phase_regressor")].load_weights("weights/sequential_training_phase_regressor_weights.h5")
model.layers[model_names.index("amplitude_regressor")].load_weights("weights/sequential_training_amplitude_regressor_weights.h5")

#save un tuned model
model.save("models/sequential_training_model.h5")

# print summary
model.summary()

# train model
history = model.fit(x_trace, [y_trace, y_phase, y_amp], epochs=500, batch_size=128, validation_split=0.2, verbose=1)

# save weights
model.layers[model_names.index("discriminator_base")].save_weights("weights/sequential_fine_tuned_discriminator_base_weights.h5")
model.layers[model_names.index("discriminator_head")].save_weights("weights/sequential_fine_tuned_discriminator_head_weights.h5")
model.layers[model_names.index("phase_regressor")].save_weights("weights/sequential_fine_tuned_phase_regressor_weights.h5")
model.layers[model_names.index("amplitude_regressor")].save_weights("weights/sequential_fine_tuned_amplitude_regressor_weights.h5")

# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf("history/sequential_fine_tuned_history.h5", key="hist")

#save tuned model
model.save("models/sequential_fine_tuned_model.h5")

2023-05-11 00:02:14.787707: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tmengel/root/lib
2023-05-11 00:02:14.787950: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-11 00:02:14.788069: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tmengel): /proc/driver/nvidia/version does not exist
2023-05-11 00:02:14.790489: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Created 261852 samples: 50.216534530956416 % pileup, 49.783465469043584 % no pileup
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 300)]          0         
                                                                 
 discriminator_base (TraceDi  (None, 2, 300)           992358    
 scriminatorBase)                                                
                                                                 
 discriminator_head (TraceDi  (None, 2, 300)           90450     
 scriminatorHead)                                                
                                                                 
Total params: 1,082,808
Trainable params: 1,082,808
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
1637/1637 [==============================] - 177s 102ms/step - loss: 0.0031 - accuracy:

KeyboardInterrupt: 